## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/drive/MyDrive/Python Projects/NLP/Fake news classifier project/Data/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.shape

(20800, 5)

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
###Drop Nan Values
df=df.dropna()


In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
y=df['label']

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.15.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [19]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [56]:
messages=X.copy()

In [57]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [58]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [59]:
messages.reset_index(inplace=True)

In [60]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [39]:
len(messages)

18285

In [33]:
len(messages['title'])

18284

In [40]:
messages['title'][7]

'Obama’s Organizing for Action Partners with Soros-Linked ‘Indivisible’ to Disrupt Trump’s Agenda'

In [61]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages['title'])):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [62]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [44]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [63]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1770, 4685, 1756, 4853, 2371, 2625, 1191, 2228, 2512, 3482],
 [4418, 2372, 4642, 3942, 3199, 1829, 528],
 [3579, 3922, 578, 779],
 [2812, 1625, 1848, 3019, 1600, 1877],
 [878, 3199, 3234, 1015, 3799, 1617, 3199, 1673, 942, 4364],
 [4449,
  1867,
  4855,
  4867,
  4472,
  2541,
  1617,
  4191,
  578,
  1088,
  1224,
  2088,
  583,
  2494,
  528],
 [2414, 3996, 41, 2009, 795, 4437, 476, 1780, 2836, 2356, 2297],
 [1139, 4506, 414, 2299, 3906, 4395, 2541, 441, 2836, 2356, 2297],
 [248, 3497, 1520, 34, 4945, 2341, 672, 467, 2541, 925],
 [1807, 4575, 3379, 4325, 2697, 2067, 4559, 4530],
 [1723, 4761, 3643, 2723, 3599, 4562, 30, 2055, 1172, 955, 4817],
 [3019, 4025, 2371, 2341, 2541, 3906],
 [614, 4928, 4303, 2913, 297, 3222, 4520, 1971, 1738],
 [762, 1930, 2303, 3550, 1675, 3587, 2994, 2836, 2356, 2297],
 [3238, 4596, 1516, 2132, 3452, 2836, 2356, 2297],
 [1947, 3718, 2506, 2317, 4651, 4844, 2012, 2476, 2428, 87],
 [2636, 2922, 2372],
 [2400, 1172, 4404, 1072, 2541, 3584, 3967, 528],
 [350

In [46]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [47]:
onehot_repr[1]

[4418, 2372, 4642, 3942, 3199, 1829, 528]

### Embedding Representation

In [64]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1770 4685 1756 ...    0    0    0]
 [4418 2372 4642 ...    0    0    0]
 [3579 3922  578 ...    0    0    0]
 ...
 [3488 2305 2342 ...    0    0    0]
 [1204 3906 2111 ...    0    0    0]
 [3310 2054 1328 ...    0    0    0]]


In [49]:
embedded_docs[1]

array([4418, 2372, 4642, 3942, 3199, 1829,  528,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [50]:
embedded_docs[0]

array([1770, 4685, 1756, 4853, 2371, 2625, 1191, 2228, 2512, 3482,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [65]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [69]:
len(embedded_docs),y.shape,X.shape

(18285, (18285,), (18285, 4))

In [71]:
len(np.array(embedded_docs))

18285

In [73]:
import numpy as np
X_final=np.array(embedded_docs)
print(len(X_final))
y_final=np.array(y)

18285


In [74]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [76]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 64ms/step - loss: 0.3183 - accuracy: 0.8397 - val_loss: 0.2056 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1373 - accuracy: 0.9466 - val_loss: 0.2164 - val_accuracy: 0.9140
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0932 - accuracy: 0.9669 - val_loss: 0.2355 - val_accuracy: 0.9140
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0651 - accuracy: 0.9775 - val_loss: 0.2937 - val_accuracy: 0.9142
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0526 - accuracy: 0.9824 - val_loss: 0.3018 - val_accuracy: 0.9168
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0358 - accuracy: 0.9878 - val_loss: 0.3998 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0284 - accuracy: 0.9905 - val_loss: 0.4627 - val_accuracy: 0.9042
E

### Adding Dropout

In [77]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [79]:
y_pred=model.predict(X_test)

189/189 [==============================] - 0s 2ms/step


In [80]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [81]:
from sklearn.metrics import confusion_matrix

In [82]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [84]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
